In [160]:
import requests
import json
from pprint import pprint
from dotenv import load_dotenv
import os
from geopy.distance import geodesic
import json
import random
from shapely.geometry import shape, Point, mapping
from shapely.ops import transform
from pyproj import CRS, Transformer
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
HOST= os.getenv("SQL_HOST")
USER= os.getenv("SQL_USER")
DATABASE= os.getenv("SQL_DATABASE")
PASSWORD=os.getenv("SQL_PASSWORD")

In [92]:

def getPlaces(latitude, longitude, api_key):
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
        "maxResultCount": 20,
        "rankPreference": "DISTANCE",
        "locationRestriction": {
            "circle": {
                "center": {
                    "latitude": latitude,
                    "longitude": longitude
                },
                "radius": 50000.0
            }
        }
    }

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.location,places.formattedAddress,places.types,places.primaryType,places.id"
    }

    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()  # Will raise an HTTPError for non-200 responses

    return response.json()

In [91]:

# Define the endpoint and payload
url = "https://places.googleapis.com/v1/places:searchNearby"

data = {
    "maxResultCount": 20,
    "rankPreference": "DISTANCE",
    "locationRestriction": {
        "circle": {
            "center": {
                "latitude": 34.861743,
                "longitude": -82.40443
            },
            "radius": 50000.0
        }
    }
}

# Headers to include specific fields
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.location,places.formattedAddress,places.types,places.primaryType,places.id"
}

# Send the request
response = requests.post(url, json=data, headers=headers)
places_data = response.json()

print(response)
print(response.text)


<Response [200]>
{
  "places": [
    {
      "id": "ChIJQ16cBsQxWIgR6TVPg9Wxfmk",
      "types": [
        "bus_stop",
        "transit_station",
        "point_of_interest",
        "establishment"
      ],
      "formattedAddress": "Greenville, SC 29609, USA",
      "location": {
        "latitude": 34.861665599999995,
        "longitude": -82.4044155
      },
      "displayName": {
        "text": "Rutherford St & W Stone Ave",
        "languageCode": "en"
      },
      "primaryType": "bus_stop"
    },
    {
      "id": "ChIJFUdKCMQxWIgRgaejy3AWcdU",
      "types": [
        "church",
        "place_of_worship",
        "point_of_interest",
        "establishment"
      ],
      "formattedAddress": "222 Rutherford St, Greenville, SC 29609, USA",
      "location": {
        "latitude": 34.8620266,
        "longitude": -82.4043009
      },
      "displayName": {
        "text": "Triune Mercy Center",
        "languageCode": "en"
      },
      "primaryType": "church"
    },
    {
   

In [4]:
pprint(places_data)

{'places': [{'displayName': {'languageCode': 'en',
                             'text': 'Rutherford St & W Stone Ave'},
             'formattedAddress': 'Greenville, SC 29609, USA',
             'location': {'latitude': 34.861665599999995,
                          'longitude': -82.4044155},
             'primaryType': 'bus_stop',
             'types': ['bus_stop',
                       'transit_station',
                       'point_of_interest',
                       'establishment']},
            {'displayName': {'languageCode': 'en',
                             'text': 'Triune Mercy Center'},
             'formattedAddress': '222 Rutherford St, Greenville, SC 29609, USA',
             'location': {'latitude': 34.8620266, 'longitude': -82.4043009},
             'primaryType': 'church',
             'types': ['church',
                       'place_of_worship',
                       'point_of_interest',
                       'establishment']},
            {'displayName': {'lang

In [112]:
#print all values
for location in places_data.get("places", []):
    address = location.get('formattedAddress', 'NULL')
    lat = location.get('location', {}).get('latitude', 'NULL')
    lon = location.get('location', {}).get('longitude', 'NULL')
    primary_type = location.get('primaryType', 'NULL')
    display_name = location.get('displayName', {}).get('text', 'NULL')

    print(f"{address}\n{lat}\n{lon}\n{primary_type}\n{display_name}\n")


Greenville, SC 29609, USA
34.861665599999995
-82.4044155
bus_stop
Rutherford St & W Stone Ave

222 Rutherford St, Greenville, SC 29609, USA
34.8620266
-82.4043009
church
Triune Mercy Center

222 Rutherford St, Greenville, SC 29609, USA
34.8620136
-82.4041982
food
Triune Mercy Center - Food Distribution Center

214 Rutherford St, Greenville, SC 29609, USA
34.8614047
-82.4040622
barbecue_restaurant
Lewis Barbecue Greenville

Greenville, SC 29609, USA
34.861143999999996
-82.404657
bus_stop
Rutherford St & Echols St

413 W Stone Ave Suite 101, Greenville, SC 29609, USA
34.8621931
-82.405002
veterinary_care
Stone Ave Vet

301 Rutherford St suite e, Greenville, SC 29609, USA
34.8621972
-82.4052147
insurance_agency
Fleming Insurance

201 Rutherford St, Greenville, SC 29609, USA
34.861001099999996
-82.40499249999999
fast_food_restaurant
Hardee’s

204 Rutherford St, Greenville, SC 29609, USA
34.86082
-82.40419
fast_food_restaurant
Arby's

307 Rutherford St, Greenville, SC 29609, USA
34.86272539

In [5]:
# Extract the list of places from the JSON
places = places_data.get("places", [])

# Filter places where the formatted address starts with a digit
filtered_places = [
    place for place in places
    if "formattedAddress" in place and place["formattedAddress"][0].isdigit()
]
for location in filtered_places:
    address = location.get('formattedAddress', 'NULL')
    lat = location.get('location', {}).get('latitude', 'NULL')
    lon = location.get('location', {}).get('longitude', 'NULL')
    primary_type = location.get('primaryType', 'NULL')
    display_name = location.get('displayName', {}).get('text', 'NULL')

    print(f"{address}\n{lat}\n{lon}\n{primary_type}\n{display_name}\n")


222 Rutherford St, Greenville, SC 29609, USA
34.8620266
-82.4043009
church
Triune Mercy Center

222 Rutherford St, Greenville, SC 29609, USA
34.8620136
-82.4041982
food
Triune Mercy Center - Food Distribution Center

214 Rutherford St, Greenville, SC 29609, USA
34.8614047
-82.4040622
barbecue_restaurant
Lewis Barbecue Greenville

413 W Stone Ave Suite 101, Greenville, SC 29609, USA
34.8621931
-82.405002
veterinary_care
Stone Ave Vet

301 Rutherford St suite e, Greenville, SC 29609, USA
34.8621972
-82.4052147
insurance_agency
Fleming Insurance

201 Rutherford St, Greenville, SC 29609, USA
34.861001099999996
-82.40499249999999
fast_food_restaurant
Hardee’s

204 Rutherford St, Greenville, SC 29609, USA
34.86082
-82.40419
fast_food_restaurant
Arby's

307 Rutherford St, Greenville, SC 29609, USA
34.862725399999995
-82.4048583
chiropractor
The Cagle Center Of Wellness

329 W Stone Ave, Greenville, SC 29609, USA
34.862031
-82.4031901
NULL
Giraffe Web Development & Design | Greenville SC

413 

In [26]:
print(places_data.get("places", [])[1].get('displayName', {}).get('text', 'NULL'))
print(places_data.get("places", [])[2].get('displayName', {}).get('text', 'NULL'))

Triune Mercy Center
Triune Mercy Center - Food Distribution Center


In [22]:
#get furthest point (radius of area)
reference_point = (34.861743, -82.40443)

# Assume places_data is already defined from your Places API v1 response
location_array = []
for i in places_data['places']:
    lat = i['location']['latitude']
    lng = i['location']['longitude']
    location_array.append((lat, lng))

# Calculate distances from reference point
distances = [
    (geodesic(reference_point, loc).meters, loc)
    for loc in location_array
]

# Find the furthest point
max_distance, furthest_point = max(distances, key=lambda x: x[0])

print(f"Furthest point: {furthest_point}")
print(f"Distance: {max_distance:} meters")

Furthest point: (34.860597299999995, -82.405853)
Distance: 181.8972281172954 meters


In [187]:
#define the area of the circle created in places_data
"""def get_max_distance(lat, lon, places_data):
    reference_point = (lat, lon)
    
    distances = [
        geodesic(reference_point, (place['location']['latitude'], place['location']['longitude'])).meters
        for place in places_data['places']
    ]
    
    return max(distances) if distances else 0
"""
def get_max_distance(lat, lon, places_data):
    reference_point = (lat, lon)

    max_distance = -1
    furthest_place_name = "Unknown"
    
    for place in places_data.get('places', []):
        try:
            place_location = (place['location']['latitude'], place['location']['longitude'])
            distance = geodesic(reference_point, place_location).meters
            
            if distance > max_distance:
                max_distance = distance
                furthest_place_name = place.get('displayName', {}).get('text', 'Unnamed place')

        except KeyError:
            continue  # Skip places with missing coordinates

    #print(f"📍 Furthest place: {furthest_place_name} ({max_distance:.2f} meters away)")
    return max_distance

In [184]:
#add websites to places
def get_websites(filtered_places):
        # Setup Chrome
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Uncomment if you want headless mode
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 1)
    
    #for location in places_data.get("places", []):
    for location in filtered_places:
        # Open the target URL
        driver.get("https://www.google.com/maps/place/")
        
        businessAddress=location.get('formattedAddress', 'NULL')
        
        #print(businessAddress)
        # Clear any pre-filled text and enter your query
        if not businessAddress[0].isdigit():
            #print("not an address")
            continue
            
    
        search_input = wait.until(EC.visibility_of_element_located((By.ID, "searchboxinput")))
    
        search_input.send_keys(businessAddress)
        
        search_input.send_keys(Keys.ENTER)
    
        businessName=location.get('displayName', {}).get('text', 'NULL')
        #print(businessName)
        xpath_selector = f"//button[@aria-label='{businessName}']"
        
        
        # Wait for button to be clickable
        try:
            button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_selector)))
            button.click()
        except:
            search_input.clear()
            search_input.send_keys(businessName)
            search_input.send_keys(Keys.ENTER)
            """
            print("no button click")
            continue
            """
        
        # Click the button
    
        
        
        # Locate the element using the 'data-tooltip' attribute
        try:
            # Sanitize businessName for XPath by escaping double quotes
            safe_business_name = businessName.replace('"', '&quot;')
            
            # XPath that matches either the "Open website" button or a labeled link like "Visit XYZ's website"
            xpath = f"""//a[
                (contains(@class, 'CsEnBe') and @data-tooltip='Open website') or 
                @aria-label="Visit {safe_business_name}'s website"
            ]"""
            
            element = wait.until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )
            """
            element = wait.until(
                EC.element_to_be_clickable((
                    By.XPATH,
                    "//a[contains(@class, 'CsEnBe') and @data-tooltip='Open website']"
                ))
            )"""
        except:
            #print("no website")
            location['website']=None
            continue
        
        
        # Get the href attribute
        href_value = element.get_attribute('href')
        
        # Print the href value
        #print("Website URL:", href_value)
        location['website']=href_value
    #Close the driver
    driver.quit()
    return filtered_places

In [ ]:
pprint(filtered_places)

In [ ]:

# -------------------------------
# STEP 2: Setup Selenium
# -------------------------------

options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Uncomment if you want headless mode
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 4)

# -------------------------------
# STEP 3: Scrape Website URLs
# -------------------------------

for location in filtered_places:
    business_address = location.get("formattedAddress", "")
    business_name = location.get("displayName", {}).get("text", "")

    print(f"Searching: {business_name} - {business_address}")

    try:
        driver.get("https://www.google.com/maps/place/")
        
        # Wait for search box and enter the address
        search_input = wait.until(EC.visibility_of_element_located((By.ID, "searchboxinput")))
        search_input.clear()
        search_input.send_keys(business_address)
        search_input.send_keys(Keys.ENTER)

        # Wait for results to load

        # Try to find website link
        try:
            website_button = wait.until(EC.element_to_be_clickable((
                By.XPATH,
                "//a[contains(@class, 'CsEnBe') and @data-tooltip='Open website']"
            )))
            website_url = website_button.get_attribute("href")
            location["website"] = website_url
            print("✔ Website found:", website_url)
        except:
            location["website"] = None
            print("✖ No website found")

    except Exception as e:
        location["website"] = None
        print(f"⚠ Error searching for {business_address}: {e}")



# -------------------------------
# STEP 5: Close Selenium
# -------------------------------

driver.quit()


In [38]:
#removes unwanted datatypes such as bus stops, parks, etc. Also removes POI and establishment from types list
def clean_places_data(places_data):
    #remove non-businesses
    filtered_places=[
    place for place in places_data.get("places", [])
    if "formattedAddress" in place and place["formattedAddress"][0].isdigit()]
    #remove POI and establishment from types data
    for place in filtered_places:
        if 'types' in place:
            place['types'] = [t for t in place['types'] if t not in ['point_of_interest', 'establishment']]
    return filtered_places

In [ ]:
filterTest=clean_places_data(response.json())
pprint(filterTest)

In [76]:
def calculate_geojson_area(geojson_file):
    # Load the GeoJSON
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    total_area_m2 = 0.0

    for feature in data['features']:
        geom = shape(feature['geometry'])

        # Get centroid for local projection (UTM)
        centroid = geom.centroid
        lon, lat = centroid.x, centroid.y

        # Determine UTM zone
        utm_zone = int((lon + 180) / 6) + 1
        is_northern = lat >= 0

        crs_wgs84 = CRS("EPSG:4326")
        crs_utm = CRS.from_proj4(
            f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
        )

        # Create projection function
        project = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform

        # Project geometry to meters (UTM) and calculate area
        geom_projected = transform(project, geom)
        total_area_m2 += geom_projected.area

        return total_area_m2  # square meters


In [89]:
def get_random_point_within_geojson(geojson_file):
    # Load the GeoJSON
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    # Combine all geometries
    combined_geom = shape(data['features'][0]['geometry'])
    for feature in data['features'][1:]:
        combined_geom = combined_geom.union(shape(feature['geometry']))

    # Bounding box for sampling
    minx, miny, maxx, maxy = combined_geom.bounds

    # Try generating a point inside the polygon
    for _ in range(10000):
        lat = random.uniform(minx, maxx)
        lon = random.uniform(miny, maxy)
        point = Point(lat, lon)
        if combined_geom.contains(point):
            return [point.y, point.x]  # [lat,lon]

    raise ValueError("Couldn't find a point inside the GeoJSON geometry.")

In [189]:
def remove_overlap_with_circle(geojson_number, lat, lon, radius_meters):
    # Determine UTM zone
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Set up projection transformers
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create circle in UTM
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)

    # Load input GeoJSON and next highest incase there is overlap at the edges
    for i in range(geojson_number, geojson_number + 2):
        with open(rf"C:\Users\Joe\Desktop\geojsons\circle{i}.geojson", 'r') as f:
            data = json.load(f)
    
        updated_features = []
        for feature in data.get("features", []):
            geom = shape(feature["geometry"])
            geom_utm = transform(to_utm, geom)
    
            # Remove overlap if present
            if geom_utm.intersects(circle_utm):
                diff = geom_utm.difference(circle_utm)
                if not diff.is_empty:
                    new_geom = transform(to_wgs84, diff)
                    updated_features.append({
                        "type": "Feature",
                        "geometry": mapping(new_geom),
                        "properties": feature.get("properties", {})
                    })
            else:
                updated_features.append(feature)
    
        # Overwrite the input file
        updated_geojson = {
            "type": "FeatureCollection",
            "features": updated_features
        }
    
        with open(rf"C:\Users\Joe\Desktop\geojsons\circle{geojson_number}.geojson", 'w') as f:
            json.dump(updated_geojson, f, indent=2)

    #print(f"Overlapping areas removed and geojson{geojson_number} has been updated.")


In [188]:
#create initial database
conn = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)

cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS businesses (
    business_id INT AUTO_INCREMENT PRIMARY KEY,
    business_name VARCHAR(255),
    primary_type VARCHAR(255),
    website VARCHAR(255),
    business_address VARCHAR(255),
    coordinates POINT NOT NULL,
    google_id VARCHAR(255) UNIQUE,
    SPATIAL INDEX(coordinates)
);
"""
create_business_types_query="""
CREATE TABLE IF NOT EXISTS business_types (
    id INT AUTO_INCREMENT PRIMARY KEY,
    google_id VARCHAR(255),
    type VARCHAR(100),
    FOREIGN KEY (google_id) REFERENCES businesses(google_id)
);
"""
create_survey_area_query="""
CREATE TABLE IF NOT EXISTS mapping_progress(
    id INT AUTO_INCREMENT PRIMARY KEY,
    circle_number INT,
    radius FLOAT,
    center POINT NOT NULL,
    focal_point VARCHAR(100),
    percent_mapped FLOAT,
    SPATIAL INDEX(center)
);
"""
cursor.execute(create_table_query)
conn.commit()
cursor.execute(create_business_types_query)
conn.commit()
cursor.execute(create_survey_area_query)
conn.commit()


cursor.close()
conn.close()

In [145]:
#upload data from places API call to sql
def sql_upload_data(filtered_places, HOST, USER, PASSWORD, DATABASE):
    conn = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
    )
    cursor = conn.cursor()
    
    business_data = []
    types_data=[]
    # Prepare business data
    for place in filtered_places:
        business_name = place.get('displayName', {}).get('text', '')
        primary_type = place.get('primaryType')
        if primary_type==None and place.get('types')!=None and place.get('types')!=[]:
            primary_type=place.get('types')[0]
        website = place.get('website')
        business_address = place.get('formattedAddress')
        latitude = place.get('location', {}).get('latitude', None)
        longitude = place.get('location', {}).get('longitude', None)
        google_id = place.get('id')  # This should be available in your data
            
        business_data.append((
            business_name,
            primary_type, 
            website,
            business_address,
            longitude,
            latitude,
            google_id
        ))

        #get all additional types
        if place.get('types')!=None and place.get('types')!=[]:
            for type_name in place['types']:
                types_data.append((
                    google_id,
                    type_name
                ))

    insert_business_query = """
        INSERT IGNORE INTO businesses (business_name, primary_type, website, business_address, coordinates, google_id)
        VALUES (%s, %s, %s, %s, POINT(%s, %s), %s)
    """
    insert_types_query= """
        INSERT IGNORE INTO business_types (google_id, type)
        VALUES (%s, %s)
    """
    cursor.executemany(insert_business_query, business_data)
    conn.commit()
    cursor.executemany(insert_types_query, types_data)
    conn.commit()
    
    #print(f"Prepared {len(business_data)} businesses for insertion")


In [73]:
def collect_places_data(current_circle_number, focal_point):
    #get the current area of the circle
    current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
    #get the calculated maximum area of the circle
    with open(r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json") as f:
        areas = json.load(f)
    max_area=areas[f"circle{current_circle_number}.geojson"]

    #loop until 97% of circle has been mapped
    while (current_area/max_area)>0.03:
        #generate a random point that has not been mapped
        randomPoint=get_random_point_within_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
        #get google places data from that random point
        places_data=getPlaces(randomPoint[0],randomPoint[1], API_KEY)
        #get the size of the circle created from the google places data
        places_radius=get_max_distance(randomPoint[0],randomPoint[1], places_data)
        #filter the places data for non-businesses
        places_data=clean_places_data(places_data)
        #use selenium to add websites to places_data
        filtered_places=get_websites(filtered_places)
        #add places to database
        sql_upload_data(filtered_places, conn, cursor)
        #modify geojson to remove the area
        remove_overlap_with_circle(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson", randomPoint[0], randomPoint[1], places_radius)
        #calculate new area
        current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
        #calculate current percentage mapped
        percentage_mapped=(current_area/max_area)
        #add point, radius, and new percentage mapped to database
        sql_upload_mapping_data(current_circle_number, places_radius, randomPoint[1],randomPoint[0], focal_point, percentage_mapped)
        

In [171]:
def sql_upload_mapping_data(circle_number, radius, longitude, latitude, focal_point, percent_mapped, HOST, USER, PASSWORD, DATABASE):
    conn = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
    )
    cursor = conn.cursor()

    insert_mapping_query = """
    INSERT IGNORE INTO mapping_progress (circle_number, radius, center, focal_point, percent_mapped)
    VALUES (%s, %s, POINT(%s, %s), %s, %s)
    """
    values = (circle_number, radius, longitude, latitude, focal_point, percent_mapped)
    cursor.execute(insert_mapping_query, values)
    conn.commit()


In [62]:
test_data=sql_upload_data((get_websites(clean_places_data(response.json()))), conn, cursor)
pprint (test_data)

222 Rutherford St, Greenville, SC 29609, USA
222 Rutherford St, Greenville, SC 29609, USA
214 Rutherford St, Greenville, SC 29609, USA
413 W Stone Ave Suite 101, Greenville, SC 29609, USA
301 Rutherford St suite e, Greenville, SC 29609, USA
201 Rutherford St, Greenville, SC 29609, USA
204 Rutherford St, Greenville, SC 29609, USA
no website
307 Rutherford St, Greenville, SC 29609, USA
329 W Stone Ave, Greenville, SC 29609, USA
413 W Stone Ave #102, Greenville, SC 29609, USA
332 W Stone Ave, Greenville, SC 29609, USA
no website
323 W Stone Ave, Greenville, SC 29609, USA
328 W Stone Ave, Greenville, SC 29609, USA
105 Rutherford St, Greenville, SC 29609, USA
100 Pete Hollis Blvd, Greenville, SC 29601, USA
320 W Stone Ave, Greenville, SC 29609, USA
Prepared 16 businesses for insertion
[('Triune Mercy Center',
  'church',
  'http://triunemercy.org/',
  '222 Rutherford St, Greenville, SC 29609, USA',
  -82.4043009,
  34.8620266,
  'ChIJFUdKCMQxWIgRgaejy3AWcdU'),
 ('Triune Mercy Center - Food 

In [177]:
def collect_places_data_test(current_circle_number, focal_point, api_key, HOST, USER, PASSWORD, DATABASE):
    #get the current area of the circle
    current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
    #get the calculated maximum area of the circle
    with open(r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json") as f:
        areas = json.load(f)
    max_area=areas[f"circle{current_circle_number}.geojson"]
        #generate a random point that has not been mapped
    randomPoint=get_random_point_within_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
        #get google places data from that random point
    places_data=getPlaces(randomPoint[0],randomPoint[1], api_key)
        #get the size of the circle created from the google places data
    places_radius=get_max_distance(randomPoint[0],randomPoint[1], places_data)
        #filter the places data for non-businesses
    filtered_places=clean_places_data(places_data)
        #use selenium to add websites to places_data
    filtered_places=get_websites(filtered_places)
        #add places to database
    sql_upload_data(filtered_places, HOST, USER, PASSWORD, DATABASE)
        #modify geojson to remove the area
    remove_overlap_with_circle(current_circle_number, randomPoint[0], randomPoint[1], places_radius)
        #calculate new area
    current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
        #calculate current percentage mapped
    percentage_mapped=(current_area/max_area)
        #add point, radius, and new percentage mapped to database
    sql_upload_mapping_data(current_circle_number, places_radius, randomPoint[1],randomPoint[0], focal_point, percentage_mapped, HOST, USER, PASSWORD, DATABASE)
    print(f"{randomPoint[0]}, {randomPoint[1]}, {places_radius}")
    print(f"percentage: {percentage_mapped}")
        

In [195]:
def collect_places_data_loop_49(focal_point, api_key, HOST, USER, PASSWORD, DATABASE):
    counter=0
    for i in range(1, 50):
        #get the current area of the circle
        current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{i}.geojson")
        #get the calculated maximum area of the circle
        with open(r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json") as f:
            areas = json.load(f)
        max_area=areas[f"circle{i}.geojson"]

        #loop until 97% of circle has been mapped
        while (current_area/max_area)>0.03:
                #generate a random point that has not been mapped
            randomPoint=get_random_point_within_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{i}.geojson")
                #get google places data from that random point
            places_data=getPlaces(randomPoint[0],randomPoint[1], api_key)
                #get the size of the circle created from the google places data
            places_radius=get_max_distance(randomPoint[0],randomPoint[1], places_data)
                #filter the places data for non-businesses
            filtered_places=clean_places_data(places_data)
                #use selenium to add websites to places_data
            filtered_places=get_websites(filtered_places)
                #add places to database
            sql_upload_data(filtered_places, HOST, USER, PASSWORD, DATABASE)
                #modify geojson to remove the area
            remove_overlap_with_circle(i, randomPoint[0], randomPoint[1], places_radius)
                #calculate new area
            current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{i}.geojson")
                #calculate current percentage mapped
            percentage_mapped=(current_area/max_area)
                #add point, radius, and new percentage mapped to database
            sql_upload_mapping_data(i, places_radius, randomPoint[1],randomPoint[0], focal_point, percentage_mapped, HOST, USER, PASSWORD, DATABASE)
            counter+=1
            #print(f"{randomPoint[0]}, {randomPoint[1]}, {places_radius}")
            #print(f"percentage: {percentage_mapped}")
            if counter>9990:
                return
        

In [ ]:
collect_places_data_loop_49("Greenville", API_KEY, HOST, USER, PASSWORD, DATABASE)

In [174]:
tester=get_websites(clean_places_data(getPlaces(34.843794,-82.397254, API_KEY)))

1 Cleveland St, Greenville, SC 29601, USA
Camp Joy
no website
1 Cleveland St #202, Greenville, SC 29601, USA
Raymond James Financial Services
300 E Camperdown Way, Greenville, SC 29601, USA
Greenville Rose Society Garden
no website
3 Cleveland St, Greenville, SC 29601, USA
Clarity Counseling Services, LLC
no website
291 E Camperdown Way, Greenville, SC 29601, USA
Shaw Rose Garden
no website
200 E Camperdown Way, Greenville, SC 29601, USA
Sue Simpson Garden
no website
1 Cleveland St, Greenville, SC 29601, USA
Shaw Garden
no website
304 E Camperdown Way, Greenville, SC 29601, USA
St. Paul's Anglican Church
no website
19 Cleveland St, Greenville, SC 29601, USA
Mc Call Melissa
no website
19 Cleveland St, Greenville, SC 29601, USA
Dr. Morton T. Edwards Jr, DMD
no website
301 E Camperdown Way, Greenville, SC 29601, USA
Greenville County Schools
201 E Camperdown Way, Greenville, SC 29601, USA
Ameriprise Financial Services, Inc.
201 E Camperdown Way, Greenville, SC 29601, USA
Amy Garrison - Fi

In [1]:
import mysql.connector

# Connect to your MySQL/MariaDB server
conn = mysql.connector.connect(
    host='HOST',
    user='USER',
    password='PASSWORD',
    database='DATABASE'
)

cursor = conn.cursor()

# Fetch all data from the table
cursor.execute("SELECT * FROM mapping_progress;")
results = cursor.fetchall()



DatabaseError: 2005 (HY000): Unknown MySQL server host 'HOST' (11001)